# LangChain4j API Overview

In [1]:
%useLatestDescriptors
%use coroutines
%use langchain4j(1.0.0-beta3)

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

In [3]:
val model = OpenAiChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .temperature(0.3)
    .build()

## Chat API

In [4]:
model.chat("How do I create in Kotlin a plus function for a Point data class that adds two Points together in the form point1 + point2?")

In Kotlin, you can overload the `plus` operator for a data class by defining the `plus` function. To create a `Point` data class that supports addition using the `+` operator, you can follow these steps:

1. Define the `Point` data class with properties for the x and y coordinates.
2. Implement the `plus` operator function to add two `Point` instances.

Here's an example of how you can do this:

```kotlin
data class Point(val x: Int, val y: Int) {
    operator fun plus(other: Point): Point {
        return Point(this.x + other.x, this.y + other.y)
    }
}

fun main() {
    val point1 = Point(1, 2)
    val point2 = Point(3, 4)
    
    val result = point1 + point2
    println(result) // Output: Point(x=4, y=6)
}
```

### Explanation:
- The `Point` data class has two properties: `x` and `y`, both of type `Int`.
- The `operator fun plus(other: Point): Point` function is defined to overload the `+` operator. It takes another `Point` as a parameter and returns a new `Point` whose coordinate

## Prompts

### 1 API

In [5]:
import dev.langchain4j.data.message.SystemMessage.systemMessage
import dev.langchain4j.data.message.UserMessage.userMessage
import dev.langchain4j.model.chat.request.chatRequest

val response = model.chat(
    chatRequest {
        messages += systemMessage(
            """
            You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
        """.trimIndent()
        )
        messages += userMessage(
            """
            What foldRight does? What will be the intermidieate and the end result of the foldRight function call?
            ```kotlin
                "baba".foldRight("") { x, acc -> "${'$'}x${'$'}{acc}${'$'}{acc}" })
            ```
            """.trimIndent()
        )
    }
).aiMessage()
response

AiMessage { text = "In Kotlin, the `foldRight` function is used to accumulate a value starting from the rightmost element of a collection (or a string, in this case). It takes an initial accumulator value and a lambda function that specifies how to combine each element with the accumulator.

Here's a breakdown of the provided code:

```kotlin
val result = "baba".foldRight("") { x, acc -> "$x${acc}${acc}" }
```

### Explanation:
- The string `"baba"` is processed from right to left.
- The initial accumulator (`acc`) is an empty string `""`.
- For each character `x` in the string, the lambda function combines `x` with the current accumulator `acc` and appends `acc` again.

### Intermediate Steps:
1. Start with `acc = ""` (initial value).
2. Process the last character `'a'`:
   - `x = 'a'`, `acc = ""` → Result: `"a"`
3. Process the second last character `'b'`:
   - `x = 'b'`, `acc = "a"` → Result: `"ba"`
4. Process the third last character `'a'`:
   - `x = 'a'`, `acc = "ba"` → Result: `"a

### 2 API

In [6]:
interface AssistantFirst {
    fun chat(userMessage: String): String
}

In [7]:
import dev.langchain4j.service.AiServices

val aiService = AiServices
    .builder(AssistantFirst::class.java)
    .chatLanguageModel(model)
    .systemMessageProvider {
        """
        You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
        """.trimIndent()
    }
    .build()

In [8]:
aiService.chat("""
Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
Make sure to include all imports, but don’t use star imports.
""".trimIndent())

Here’s an idiomatic Kotlin code snippet that creates a flow of five elements and prints each one using the `collect` function:

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    createFlow().collect { value ->
        println(value)
    }
}

fun createFlow(): Flow<Int> = flow {
    for (i in 1..5) {
        emit(i)
    }
}
```

### Explanation:
- The `createFlow` function creates a flow of integers from 1 to 5 using the `flow` builder.
- The `main` function uses `runBlocking` to run a coroutine that collects the flow and prints each emitted value.
- The `collect` function is used without any arguments, as requested.

### 3 API

In [9]:
import dev.langchain4j.service.SystemMessage

interface AssistantSecond {

    @SystemMessage(
        """
        You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
        """
    )
    fun chat(userMessage: String): String
}

In [10]:
val assistant = AiServices.create(AssistantSecond::class.java, model)

In [11]:
assistant.chat("""
Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
Make sure to include all imports, but don’t use star imports.
""".trimIndent())

Here's a concise and idiomatic Kotlin code snippet that creates a flow of five elements and prints each of them using the `collect` function:

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    createFlow().collect { value ->
        println(value)
    }
}

fun createFlow(): Flow<Int> = flow {
    for (i in 1..5) {
        emit(i)
    }
}
```

### Explanation:
- **Imports**: Specific imports are used for clarity.
- **Flow Creation**: The `createFlow` function generates a flow of integers from 1 to 5.
- **Collecting Values**: The `collect` function is used to print each emitted value.
- **runBlocking**: This is used to run the coroutine in the main function, ensuring that the flow is collected properly.

## Stream

### Async

In [12]:
import dev.langchain4j.model.chat.chat

runBlocking {
    model.chat {
        messages += systemMessage("You are an expert Kotlin developer.")
        messages += userMessage(
            """
                Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
                Make sure to include all imports, but don’t use star imports.
            """.trimIndent()
        )
    }
}


ChatResponse { aiMessage = AiMessage { text = "Certainly! Below is a Kotlin program that creates a flow of five elements (1, 2, 3, 4, and 5) and prints each of them using the `collect` function without any arguments. Make sure you have the necessary dependencies for Kotlin Coroutines in your project.

```kotlin
import kotlinx.coroutines.CoroutineScope
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.launch
import kotlinx.coroutines.runBlocking

fun main() = runBlocking {
    val numberFlow: Flow<Int> = flow {
        for (i in 1..5) {
            emit(i)
        }
    }

    numberFlow.collect()
}

fun Flow<Int>.collect() {
    CoroutineScope(Dispatchers.Default).launch {
        this@collect.collect { value ->
            println(value)
        }
    }
}
```

### Explanation:
1. **Imports**: The necessary imports for coroutines and flow are included.
2. **Flow Creation**: A flow of integers from 1 t

### Flow

In [13]:
import dev.langchain4j.model.chat.StreamingChatLanguageModelReply
import dev.langchain4j.model.chat.chatFlow
import dev.langchain4j.model.openai.OpenAiStreamingChatModel
import kotlinx.coroutines.flow.Flow

val streamModel = OpenAiStreamingChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .temperature(0.3)
    .build()

val flowResponse = streamModel.chatFlow {
    messages += systemMessage("You are an expert Kotlin developer.")
    messages += userMessage(
        """
        Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
        Make sure to include all imports, but don’t use star imports.
        """.trimIndent()
    )
}

runBlocking {
    flowResponse.collect {
        when (it) {
            is StreamingChatLanguageModelReply.PartialResponse -> {
                print(it.partialResponse) // Stream output as it arrives
            }
            is StreamingChatLanguageModelReply.CompleteResponse -> {
                println("\nComplete: ${it.response.aiMessage().text()}")
            }
            is StreamingChatLanguageModelReply.Error -> {
                println("Error occurred: ${it.cause.message}")
            }
        }
    }
}

Certainly! Below is a Kotlin program that creates a flow of five elements (1, 2, 3, 4, and 5) and prints each element using the `collect` function. The necessary imports are included without using star imports.

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    val numberFlow: Flow<Int> = flow {
        for (i in 1..5) {
            emit(i) // Emit each number
        }
    }

    numberFlow.collect() { value ->
        println(value) // Print each emitted value
    }
}
```

### Explanation:
- We import the necessary classes from the `kotlinx.coroutines.flow` package.
- The `main` function is defined as a coroutine using `runBlocking` to allow the use of suspending functions.
- A `Flow` of integers is created using the `flow` builder, emitting the numbers 1 through 5.
- The `collect` function is called on the flow, which takes a lambda to process 

### AiService

In [14]:
@file:DependsOn("dev.langchain4j:langchain4j-reactor:1.0.0-beta2")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactor:1.10.1")

In [15]:
import reactor.core.publisher.Flux

interface AssistantThird {
    fun chat(message: String): Flux<String>
}

In [16]:
val assistant = AiServices.create(AssistantThird::class.java, streamModel)

In [17]:
import kotlinx.coroutines.reactive.asFlow

val response = assistant.chat("""
Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
Make sure to include all imports, but don’t use star imports.
""".trimIndent()).asFlow()

runBlocking {
    response.collect {
        print(it)
    }
}

To create a flow of five elements and print each one using the `collect` function in Kotlin, you'll need to use the Kotlin Coroutines library. Below is a complete example that includes all necessary imports and demonstrates how to achieve this:

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    // Create a flow of five elements
    val numberFlow: Flow<Int> = flow {
        for (i in 1..5) {
            emit(i) // Emit each number
        }
    }

    // Collect and print each element from the flow
    numberFlow.collect { value ->
        println(value)
    }
}
```

### Explanation:
1. **Imports**: We import the necessary classes from the `kotlinx.coroutines.flow` package.
2. **Flow Creation**: We create a flow using the `flow` builder, emitting numbers from 1 to 5.
3. **Collecting Values**: We use the `collect` function to print each emitted val

## Structured Outputs

### JSON_MODE

In [18]:
import com.fasterxml.jackson.annotation.JsonCreator
import com.fasterxml.jackson.annotation.JsonProperty

/**
 * Represents a person with basic personal attributes.
 *
 * @property name The name of the person.
 * @property age The age of the person in years.
 * @property height The height of the person in meters.
 * @property married Indicates if the person is married.
 */
data class Person @JsonCreator constructor(
    @JsonProperty("name") val name: String,
    @JsonProperty("age") val age: Int,
    @JsonProperty("height") val height: Double,
    @JsonProperty("married") val married: Boolean
)

interface PersonExtractor {
    fun extractPersonFrom(text: String): Person
}

val jsonModel = OpenAiChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .temperature(0.3)
    .responseFormat("json_schema")
    .strictJsonSchema(true)
    .build()

val personExtractor = AiServices.create(PersonExtractor::class.java, jsonModel)

In [19]:
personExtractor.extractPersonFrom(
    """
        John is 42 years old and lives an independent life.
        He stands 1.75 meters tall and carries himself with confidence.
        Currently unmarried, he enjoys the freedom to focus on his personal goals and interests.
    """
)

Person(name=John, age=42, height=1.75, married=false)

In [20]:
data class Step @JsonCreator constructor(
    @JsonProperty("explanation") val explanation: String,
    @JsonProperty("output") val output: String
)

data class MathResponse @JsonCreator constructor(
    @JsonProperty("steps") val steps: List<Step>,
    @JsonProperty("finalAnswer") val finalAnswer: String
)

interface MathAssistant {
    @SystemMessage("You are a helpful math tutor.")
    fun chat(message: String): MathResponse
}

val assistant = AiServices.create(MathAssistant::class.java, jsonModel)


In [21]:
val response = assistant.chat("solve 8x + 31 = 2")

println(response.steps.withIndex().joinToString("\n") { "${it.index + 1}. ${it.value.explanation}" })
println("Answer:")
println(response.finalAnswer)

1. Subtract 31 from both sides to isolate the term with x.
2. This simplifies to 8x = -29.
3. Now, divide both sides by 8 to solve for x.
4. This simplifies to x = -3.625.
Answer:
x = -3.625


### Function calling

In [22]:
import dev.langchain4j.agent.tool.Tool

class Tools {
    @Tool
    fun plus(a: Int, b: Int): Int = a + b

    @Tool
    fun minus(a: Int, b: Int): Int = a - b

    @Tool
    fun times(a: Int, b: Int): Int = a * b

    @Tool
    fun div(a: Int, b: Int): Int = a / b
}

interface MathAssistant {
    @SystemMessage("You are a helpful math tutor.")
    fun chat(message: String): String
}

val assistant = AiServices.builder(MathAssistant::class.java)
    .chatLanguageModel(model)
    .tools(Tools())
    .build()

In [23]:
assistant.chat("""
Solve this:
* (5 + 10) * (20 - 3) / 4
* (a + b)^2
* ax^2 + bx + c = 0 for a=1, b=-3, c=-4
*  S = n * (a1 + an) / 2
    n = 10
    a1 = 3
    an = 20
""")

Let's break down the calculations step by step:

1. **Calculate \((5 + 10) * (20 - 3) / 4\)**:
   - \(5 + 10 = 15\)
   - \(20 - 3 = 17\)
   - \(15 * 17 = 255\)
   - \(255 / 4 = 63.75\)

2. **Expand \((a + b)^2\)**:
   - The expansion is \(a^2 + 2ab + b^2\).

3. **Solve the quadratic equation \(ax^2 + bx + c = 0\) for \(a=1\), \(b=-3\), \(c=-4\)**:
   - The equation becomes \(1x^2 - 3x - 4 = 0\).
   - The roots can be found using the quadratic formula: 
     \[
     x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
     \]
   - Here, \(b^2 - 4ac = (-3)^2 - 4(1)(-4) = 9 + 16 = 25\).
   - Thus, the roots are:
     \[
     x = \frac{3 \pm 5}{2}
     \]
     - \(x_1 = \frac{8}{2} = 4\)
     - \(x_2 = \frac{-2}{2} = -1\)

4. **Calculate the sum \(S = n * (a1 + an) / 2\)**:
   - \(n = 10\), \(a1 = 3\), \(an = 20\)
   - \(S = 10 * (3 + 20) / 2 = 10 * 23 / 2 = 115\)

### Summary of Results:
- \((5 + 10) * (20 - 3) / 4 = 63.75\)
- \((a + b)^2 = a^2 + 2ab + b^2\)
- Roots of \(x^2 - 3x - 4 = 0\) are \(x = 4\